Rare diseases

In [3]:
"""1º Importo las librerías que creo que puedo necesitar en esta tarea."""

# Manejo de datos
import pandas as pd  # Análisis y manipulación de datos en tablas
import numpy as np  # Cálculos numéricos y matrices
import os

# Visualización de datos
import matplotlib.pyplot as plt  # Gráficos básicos en 2D
import seaborn as sns  # Gráficos estadísticos mejorados
import plotly.express as px  # Gráficos interactivos simplificados
import plotly.graph_objects as go  # Control avanzado de gráficos en Plotly
import plotly.io as pio  # Interfaz de entrada/salida de Plotly
import squarify  # Visualización de diagramas de "treemap"

# Web Scraping y APIs
import requests  # Peticiones HTTP para obtener datos de APIs como PubChem

# Quimioinformática y Bioinformática
from rdkit import Chem  # Manejo de estructuras químicas
from rdkit.Chem import DataStructs, AllChem  # Análisis molecular
import pubchempy as pcp  # Interacción con la base de datos de PubChem

In [4]:
carpeta_data = r"C:\Users\maria\ITAcademy\Sprint 10 - Pràctica final\Data\tablas_extraidas_Orphadata"
orphadata = {}
for archivo in os.listdir(carpeta_data):
    if archivo.endswith(".csv"):
        ruta_archivo = os.path.join(carpeta_data, archivo)
        nombre_dataframe = archivo.replace(".csv", "")
        orphadata[nombre_dataframe] = pd.read_csv(ruta_archivo)
orphadata.keys()

dict_keys(['df_diseases', 'df_disease_gene', 'df_genes', 'df_rd_classification', 'df_rd_epidemiology', 'df_rd_functional_consequences', 'df_rd_history', 'df_rd_list', 'df_rd_phenotypes'])

In [9]:
# Cambiar nombre col hchId
orphadata['df_diseases'].rename(columns={'class_hchId': 'hchId'}, inplace=True)

# Cambiar nombre col Orphacode
orphadata['df_rd_epidemiology'].rename(columns={'Code Disease': 'ORPHAcode'}, inplace=True)

# cambiar nombre gene name y gene symbol
orphadata['df_genes'].rename(columns={'name': 'Gene name'}, inplace=True)
orphadata['df_genes'].rename(columns={'symbol': 'Gene symbol'}, inplace=True)

# Cambiar nombre col Orphacode
orphadata['df_rd_functional_consequences'].rename(columns={'Código Orphanet': 'ORPHAcode'}, inplace=True)

In [10]:
for nombre_archivo, dataframe in orphadata.items():
    print(f"Archivo: {nombre_archivo}")
    print(f"Total filas = {len(dataframe)}")
    print("Columnas:", dataframe.columns.tolist())
    print("\n")

Archivo: df_diseases
Total filas = 27613
Columnas: ['hchId', 'ORPHAcode', 'preferredTerm']


Archivo: df_disease_gene
Total filas = 8193
Columnas: ['ORPHAcode', 'Disorder name', 'Disorder type', 'Gene symbol', 'Gene name', 'Gene type', 'Gene synonyms', 'Locus', 'Validation source']


Archivo: df_genes
Total filas = 4478
Columnas: ['HGNC', 'Gene name', 'Gene symbol']


Archivo: df_rd_classification
Total filas = 35
Columnas: ['hchId', 'hchTag']


Archivo: df_rd_epidemiology
Total filas = 16119
Columnas: ['Typology', 'ORPHAcode', 'Preferred term', 'Prevalence Type', 'Prevalence Value']


Archivo: df_rd_functional_consequences
Total filas = 1046
Columnas: ['Nombre', 'ORPHAcode', 'Consecuencias Funcionales']


Archivo: df_rd_history
Total filas = 6994
Columnas: ['data', 'AverageAgeOfOnset', 'Date', 'DisorderGroup', 'ORPHAcode', 'OrphanetURL', 'Preferred term', 'TypeOfInheritance', 'Typology']


Archivo: df_rd_list
Total filas = 11074
Columnas: ['ORPHAcode', 'Preferred term']


Archivo: df_

In [11]:
# Mega merge
tabla_completa = orphadata['df_diseases'] \
    .merge(orphadata['df_disease_gene'], on='ORPHAcode') \
    .merge(orphadata['df_genes'], on='Gene symbol') \
    .merge(orphadata['df_rd_classification'], on='hchId') \
    .merge(orphadata['df_rd_epidemiology'], on='ORPHAcode') \
    .merge(orphadata['df_rd_functional_consequences'], on='ORPHAcode')
print(tabla_completa.shape)
tabla_completa

(45028, 20)


,hchId,ORPHAcode,preferredTerm,Disorder name,Disorder type,Gene symbol,Gene name_x,Gene type,Gene synonyms,Locus,Validation source,HGNC,Gene name_y,hchTag,Typology,Preferred term,Prevalence Type,Prevalence Value,Nombre,Consecuencias Funcionales
0,146,324,Fabry disease,Fabry disease,Disease,GLA,galactosidase alpha,gene with protein product,['GALA'],"[{'GeneLocus': 'Xq22.1', 'LocusKey': 1}]",20301469[PMID],4296.0,galactosidase alpha,Orphanet classification of rare cardiac diseases,Disease,Fabry disease,Prevalence at birth,NaN,Fabry disease,NaN
1,146,324,Fabry disease,Fabry disease,Disease,GLA,galactosidase alpha,gene with protein product,['GALA'],"[{'GeneLocus': 'Xq22.1', 'LocusKey': 1}]",20301469[PMID],4296.0,galactosidase alpha,Orphanet classification of rare cardiac diseases,Disease,Fabry disease,Prevalence at birth,NaN,Fabry disease,NaN
2,146,324,Fabry disease,Fabry disease,Disease,GLA,galactosidase alpha,gene with protein product,['GALA'],"[{'GeneLocus': 'Xq22.1', 'LocusKey': 1}]",20301469[PMID],4296.0,galactosidase alpha,Orphanet classification of rare cardiac diseases,Disease,Fabry disease,Prevalence at birth,NaN,Fabry disease,NaN
3,146,324,Fabry disease,Fabry disease,Disease,GLA,galactosidase alpha,gene with protein product,['GALA'],"[{'GeneLocus': 'Xq22.1', 'LocusKey': 1}]",20301469[PMID],4296.0,galactosidase alpha,Orphanet classification of rare cardiac diseases,Disease,Fabry disease,Prevalence at birth,NaN,Fabry disease,NaN
4,146,324,Fabry disease,Fabry disease,Disease,GLA,galactosidase alpha,gene with protein product,['GALA'],"[{'GeneLocus': 'Xq22.1', 'LocusKey': 1}]",20301469[PMID],4296.0,galactosidase alpha,Orphanet classification of rare cardiac diseases,Disease,Fabry disease,Prevalence at birth,NaN,Fabry disease,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45023,234,357027,Hereditary retinoblastoma,Hereditary retinoblastoma,Clinical subtype,MYCN,"MYCN proto-oncogene, bHLH transcription factor",gene with protein product,"['MYCNOT', 'N-myc', 'bHLHe37']","[{'GeneLocus': '2p24.3', 'LocusKey': 1}]",23498719[PMID]_23498720[PMID],7559.0,"MYCN proto-oncogene, bHLH transcription factor",Classification of pediatric cancer (Functional),Clinical subtype,Hereditary retinoblastoma,Point prevalence,NaN,Hereditary retinoblastoma,NaN
45024,234,357034,Non-hereditary retinoblastoma,Non-hereditary retinoblastoma,Clinical subtype,RB1,RB transcriptional corepressor 1,gene with protein product,"['PPP1R130', 'RB', 'prepro-retinoblastoma-asso...","[{'GeneLocus': '13q14.2', 'LocusKey': 1}]",23498719[PMID]_23498780[PMID]_16934146[PMID],9884.0,RB transcriptional corepressor 1,Classification of pediatric cancer (Functional),Clinical subtype,Non-hereditary retinoblastoma,Annual incidence,NaN,Non-hereditary retinoblastoma,NaN
45025,234,357034,Non-hereditary retinoblastoma,Non-hereditary retinoblastoma,Clinical subtype,RB1,RB transcriptional corepressor 1,gene with protein product,"['PPP1R130', 'RB', 'prepro-retinoblastoma-asso...","[{'GeneLocus': '13q14.2', 'LocusKey': 1}]",23498719[PMID]_23498780[PMID]_16934146[PMID],9884.0,RB transcriptional corepressor 1,Classification of pediatric cancer (Functional),Clinical subtype,Non-hereditary retinoblastoma,Point prevalence,NaN,Non-hereditary retinoblastoma,NaN
45026,234,357034,Non-hereditary retinoblastoma,Non-hereditary retinoblastoma,Clinical subtype,MYCN,"MYCN proto-oncogene, bHLH transcription factor",gene with protein product,"['MYCNOT', 'N-myc', 'bHLHe37']","[{'GeneLocus': '2p24.3', 'LocusKey': 1}]",23498719[PMID]_23498720[PMID],7559.0,"MYCN proto-oncogene, bHLH transcription factor",Classification of pediatric cancer (Functional),Clinical subtype,Non-hereditary retinoblastoma,Annual incidence,NaN,Non-hereditary retinoblastoma,NaN
